In [1]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions_assignment2 import*

In [2]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "inst_competition/" #"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur049_n_300_m_17695" #"heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [3]:
### GA ###
def GA(pop_size, init_no_plexes, node_impact_orig, node_degree_orig, edge_assignment_orig, edge_weights, s):
    ### make population
    population = generate_rand_pop(pop_size, init_no_plexes, node_impact_orig, 
                                    node_degree_orig, edge_assignment_orig, edge_weights, s)
    
    t = 0
    while t < 5:
        print("generation", t)
        ### subsequently select 2 parents to create 2 children
        weights = [solution.fitness for solution in population]
        children = []
        for i in range(25): # create 25*2 children
            # Choose parents based on weights
            parents = random.choices(population, weights=weights, k=2)
            ### recombine to generate children
            # create children assignments
            child1 = recombine(parents[0].plex_assignment, parents[1].plex_assignment, node_impact_orig, edge_weights, s)
            child2 = recombine(parents[1].plex_assignment, parents[0].plex_assignment, node_impact_orig, edge_weights, s)
            ### mutate children

            ### evaluate
            child1 = assignment_to_solution(child1, node_impact_orig, node_degree_orig, edge_weights, edge_assignment_orig, s)
            child2 = assignment_to_solution(child2, node_impact_orig, node_degree_orig, edge_weights, edge_assignment_orig, s)

            children.append(child1)
            children.append(child2)

        ### replace
        sorted_parents = sorted(population, key=lambda x: x.score, reverse = False)
        # Selecting top solutions based on score
        population = sorted_parents[:50]
        for p in population:
            print(p.score)
        population = population + children
        t = t+1
    
    # get the best solution
    sorted_parents = sorted(population, key=lambda x: x.score, reverse = False)
    # Selecting top solutions based on score
    return(sorted_parents[0])

# Try it Out

In [4]:
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)

In [5]:
start = time.time()
test = GA(100, 10, node_impact_orig, node_degree_orig, edge_assignment_orig, edge_weights, s)
print(time.time()-start)

generation 0
50898.0
50979.0
50996.0
51005.0
51008.0
51120.0
51125.0
51131.0
51162.0
51162.0
51163.0
51167.0
51172.0
51182.0
51218.0
51225.0
51227.0
51255.0
51281.0
51290.0
51318.0
51320.0
51331.0
51331.0
51331.0
51332.0
51344.0
51347.0
51355.0
51366.0
51373.0
51378.0
51380.0
51381.0
51387.0
51392.0
51397.0
51406.0
51411.0
51423.0
51428.0
51430.0
51430.0
51438.0
51443.0
51450.0
51457.0
51462.0
51465.0
51467.0
generation 1
36722.0
39153.0
39865.0
39986.0
40120.0
40427.0
40551.0
40582.0
40961.0
41056.0
41075.0
41098.0
41272.0
41536.0
41617.0
41733.0
41920.0
41964.0
42034.0
42077.0
42091.0
42307.0
42397.0
42474.0
42499.0
42523.0
42534.0
42976.0
43171.0
43232.0
43438.0
43768.0
43825.0
43861.0
44183.0
45998.0
46120.0
46383.0
46392.0
46406.0
46527.0
46547.0
46847.0
47286.0
47449.0
47603.0
48001.0
48511.0
49254.0
50898.0
generation 2
34929.0
34941.0
35184.0
35217.0
35439.0
35828.0
36322.0
36707.0
36722.0
37183.0
37490.0
38761.0
39153.0
39300.0
39381.0
39448.0
39609.0
39865.0
39986.0
40016.0
4

In [6]:
print(test.score)
print(test.fitness)

30268.0
100220.0


In [7]:
write_solution(test.edge_assignment, problem_instance, edge_weights, edges_n1, edges_n2, algorithm = "GA")